In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## Test "Image"

In [3]:
input_shape = np.array([1, 160, 160, 3])
image = np.random.random_sample(input_shape)

## Original Model

In [33]:
original_model = tf.keras.models.load_model("./saved_model/"+'facenet_keras.h5', compile=False)

print("== Input details ==")
print("shape:", original_model.input.shape)
print("type:", original_model.input.dtype)
print("\n== Output details ==")
print("shape:", original_model.output.shape)
print("type:", original_model.output.dtype)

== Input details ==
shape: (None, 160, 160, 3)
type: <dtype: 'float32'>

== Output details ==
shape: (None, 128)
type: <dtype: 'float32'>


In [15]:
original_model_output = original_model.predict(image)[0]

## TFLite converted Model

In [8]:
tf_lite_model = tf.lite.Interpreter(model_path='./facenet.tflite')
tf_lite_model.allocate_tensors()

input_details = tf_lite_model.get_input_details()
output_details = tf_lite_model.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 160 160   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [  1 128]
type: <class 'numpy.float32'>


In [13]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(image, dtype=np.float32)
tf_lite_model.set_tensor(input_details[0]['index'], input_data)

tf_lite_model.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
tf_lite_model_output = tf_lite_model.get_tensor(output_details[0]['index'])

In [19]:
(original_model_output - tf_lite_model_output).sum()

1.9185245e-07